In [ ]:
import os
import numpy as np 
import pandas as pd 

from keras.preprocessing import image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_df = pd.read_csv("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv")
train_df.head()

In [ ]:
train_df['Class'].value_counts()

In [ ]:
y=train_df['Class']
train_df['Kfold']=-1

kf=StratifiedKFold(n_splits=5)
for f,(t_,v_) in enumerate(kf.split(X=train_df,y=y)):
    train_df.loc[v_,'Kfold']=f
    
train_df.head()

In [ ]:
train = train_df[train_df['Kfold']!=4]
train['Class'].value_counts()

In [ ]:
valid = train_df[train_df['Kfold']==4]
valid['Class'].value_counts()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range = 30,
                                     shear_range=0.2,
                                     zoom_range=0.4,
                                     horizontal_flip=True,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2)

train_generator=train_datagen.flow_from_dataframe(dataframe=train,
                                            directory="../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train",
                                            x_col="Image",
                                            y_col="Class",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(331,331))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator=valid_datagen.flow_from_dataframe(dataframe=valid,
                                                    directory="../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train",
                                                    x_col="Image",
                                                    y_col="Class",
                                                    subset="training",
                                                    batch_size=128,
                                                    seed=42,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(331,331))

In [ ]:
from keras.applications import Xception
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical

pretrain_model = Xception( include_top=False, 
                            weights='imagenet',
                            input_shape=[331,331,3])

for layer in pretrain_model.layers[:-8]:
    layer.trainable = False
    
x = Flatten()(pretrain_model.output)
prediction = Dense(6,activation='softmax')(x)
my_model = Model(inputs=pretrain_model.input, outputs=prediction)

from keras.optimizers import Adam
adam=Adam(learning_rate=0.001)

my_model.compile(optimizer='adam',loss ="categorical_crossentropy",metrics=["accuracy"])

history = my_model.fit(train_generator,
                       steps_per_epoch=5176//128,
                       validation_data=valid_generator,
                       validation_steps=1293//128,
                       epochs=16)

In [ ]:
my_model.save('model.h5')

In [ ]:
name=[]
y_pred=[]
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
s=0
for i in os.listdir('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'):
    name.append(i)
    i='../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'+i
    img=image.load_img(i,target_size=(331,331,3))
    img=image.img_to_array(img)/255
    pred=my_model.predict(img.reshape(1,331,331,3))
    y_pred.append(labels[np.argmax(pred[0])])
    s+=1
    if s%100==0:
        print(s)
    
data=pd.DataFrame((zip(name,y_pred)),columns=['Image','Class'])
data.head()

In [ ]:
data.to_csv('submission.csv',index=False)
data.shape